In [5]:
from utils import read_config, OracleAgent, MySQLAgent
import re
import pandas as pd
import os
from py2neo import Graph, Node, Relationship
from langchain.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

from langchain.chat_models  import AzureChatOpenAI

import google.generativeai as genai
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

import prompts
import ast

In [2]:
from utils import read_config, OracleAgent, MySQLAgent
import os
configs = read_config(".env/info.json")

os.environ["GOOGLE_API_KEY"] = configs['gkey']

O_KEY = configs['okey']
os.environ["OPENAI_API_KEY"] = O_KEY
DWDB = configs['DW_conn_info']
dw_agent = OracleAgent(DWDB)

In [3]:
mysql_configs = configs["Crawler_mysql_conn_info_BIData"]
mysql_agent = MySQLAgent(mysql_configs)

aoai_configs = configs['AOAI']

In [4]:
OPENAI_API_BASE = aoai_configs['OPENAI_API_BASE']
OPEN_AI_VERSION = aoai_configs['OPEN_AI_VERSION']
GPT_DEPLOYMENT_NAME = aoai_configs['GPT_DEPLOYMENT_NAME']
OPENAI_API_KEY = aoai_configs['OPENAI_API_KEY']
OPENAI_API_TYPE = aoai_configs['OPENAI_API_TYPE']

llm = AzureChatOpenAI(
    azure_endpoint=OPENAI_API_BASE,
    openai_api_version=OPEN_AI_VERSION,
    azure_deployment=GPT_DEPLOYMENT_NAME,
    openai_api_key=OPENAI_API_KEY,
    openai_api_type="azure",
)

# llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key = O_KEY)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\bibot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


#### add one complicated case

In [5]:
# rela = """
# Datasource = [
#     'apps.mtl_material_transactions',
#     'apps.oe_order_headers_all',
#     'apps.mtl_transaction_types',
#     'org_organization_definitions',
#     'apps.oe_order_lines_all',
#     'apps.oe_transaction_types_tl',
#     'apps.mtl_system_items_b',
#     'apps.mtl_item_categories_v',
#     'apps.ar_customers_all_v',
#     'xxom_quote_margin',
#     'xxinv_items_upload_t1',
#     'apps.ra_customer_trx_all',
#     'apps.ra_customer_trx_lines_all',
#     'apps.ra_cust_trx_types_all',
#     'apps.hz_cust_accounts',
#     'xxtcpq_pcg_avcost',
#     'hr_operating_units'
# ]\n\n
# Relationship = [
#     (JOIN: apps.oe_order_headers_all.header_id = apps.oe_order_lines_all.header_id),
#     (JOIN: apps.mtl_material_transactions.source_line_id = apps.oe_order_lines_all.line_id),
#     (JOIN: apps.mtl_material_transactions.transaction_type_id = apps.mtl_transaction_types.transaction_type_id),
#     (JOIN: org_organization_definitions.organization_id = apps.oe_order_headers_all.org_id),
#     (JOIN: org_organization_definitions.organization_id = apps.mtl_system_items_b.organization_id),
#     (JOIN: apps.mtl_system_items_b.inventory_item_id = apps.oe_order_lines_all.inventory_item_id),
#     (JOIN: apps.mtl_system_items_b.inventory_item_id = apps.mtl_item_categories_v.inventory_item_id),
#     (JOIN: apps.mtl_item_categories_v.organization_id = org_organization_definitions.organization_id),
#     (JOIN: apps.ar_customers_all_v.customer_id = apps.oe_order_headers_all.sold_to_org_id),
#     (JOIN: apps.hz_cust_accounts.cust_account_id = apps.ar_customers_all_v.customer_id),
#     (JOIN: apps.ra_customer_trx_lines_all.customer_trx_id = apps.ra_customer_trx_all.customer_trx_id),
#     (JOIN: apps.ra_customer_trx_lines_all.inventory_item_id = apps.mtl_system_items_b.inventory_item_id),
#     (JOIN: apps.ra_cust_trx_types_all.cust_trx_type_id = apps.ra_customer_trx_lines_all.cust_trx_type_id),
#     (JOIN: apps.ra_cust_trx_types_all.org_id = org_organization_definitions.operating_unit),
#     (JOIN: xxtcpq_pcg_avcost.ebs_so_header_id = apps.oe_order_headers_all.header_id),
#     (JOIN: xxom_quote_margin.quote_number = apps.oe_order_headers_all.order_number),
#     (JOIN: xxinv_items_upload_t1.segment1 = apps.mtl_system_items_b.segment1),
#     (Filter: apps.mtl_item_categories_v.category_set_name = 'YFY Inventory'),
#     (Filter: apps.mtl_item_categories_v.enabled_flag = 'Y'),
#     (Filter: apps.oe_transaction_types_tl.LANGUAGE = 'US'),
#     (Filter: org_organization_definitions.organization_code NOT IN ('AEG')),
#     (Filter: apps.oe_order_lines_all.item_type_code <> 'MODEL'),
#     (Filter: apps.oe_order_lines_all.flow_status_code = 'CLOSED'),
#     (Filter: apps.mtl_material_transactions.subinventory_code = 'STAGE')
# ]

# """


In [6]:
# view_lineage.at[2, 'view_name'] = 'XXBI_SALES_DETAILS_PCG_TW'
# view_lineage.at[2, 'text'] = text_sql
# view_lineage.at[2, 'lineage'] = rela


In [7]:
# mysql_agent.write_table(data=view_lineage, table_name='data_lineage', if_exists='replace', data_type=None, index=None)

## BIDB

In [8]:
BIDB = configs['BIDB_conn_info']
bidb_agent = OracleAgent(BIDB)

In [9]:
query = """
SELECT view_name, text FROM ALL_Views
where owner = 'ODS' AND rownum <= 5
"""

bidb_view = bidb_agent.read_table(query=query)

In [42]:
bidb_view['text'] = bidb_view['text'].str.replace('\n', ' ')
bidb_view['text'] = bidb_view['text'].str.replace('\t', ' ')
bidb_view['source'] = bidb_view['text'].apply(lambda x: re.search(r'FROM\s+(.*)', x, re.DOTALL | re.IGNORECASE).group(0))
# bidb_view['source'] = bidb_view['source'].apply(lambda x: x.split('UNION' if 'UNION' in x else 'union')[0].strip())
bidb_view['source'] = bidb_view['source'].apply(lambda x:re.sub(r'--.*?(\s+|$)', '', x)) 
bidb_view['source'] = bidb_view['source'].apply(lambda x:re.sub(r'1=1.*?(\s+|$)', '', x)) 
bidb_view['lineage'] = ''

In [44]:
bidb_view

,view_name,text,source,lineage
0,C$_0W_YFY_AV_TW_R,"select ""C1_ROW_ID"",""C2_ORG_ID"",""C3_REF_AV_HEAD...",from ( select W_YFY_AV_TW_R.ROW_ID C1_ROW...,
1,C$_0W_YFY_IND_FIN_INFO_FS,"select FIN_INFO.SEQ C1_SEQ, FIN_INFO.ACC...",from ODS.TC$_0W_YFY_IND_FIN_INFO_FS FIN_INFO,
2,OP_FACT_CHP_INVENTORY_ETH_PULP,"SELECT PERIOD_NAME,STOCK_DATE TDATE,ORG_CODE,'...",FROM W_FACTORY_INV_BALANCE_F UNION ALL SELECT ...,
3,OP_FACT_CHP_INVENTORY_REDEFINE,"SELECT PERIOD_NAME,TDATE,ORG_CODE ,CASE ORG_...","FROM ( SELECT F.PERIOD_NAME,F.BALANCE_DATE T...",
4,OP_FACT_CHP_SALES_DETAILS,"SELECT '當月受訂' TYPE, --GREATEST(F.REQU...","from W_CHP_SALES_NOMANUAL_F F ,W_CHP_ORDER...",


In [45]:
system_template = prompts.SYSTEM_PROMPT
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{table_name}, {datasource}")
]

CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [46]:
chain = CHAT_PROMPT | llm
for idx, row in bidb_view.iterrows():
    input_data = {
        'table_name': row.view_name,
        'datasource': row.source
    }
    llm_response = chain.invoke(input_data)

    bidb_view.at[idx, 'lineage'] = llm_response.content

In [47]:
bidb_view

,view_name,text,source,lineage
0,C$_0W_YFY_AV_TW_R,"select ""C1_ROW_ID"",""C2_ORG_ID"",""C3_REF_AV_HEAD...",from ( select W_YFY_AV_TW_R.ROW_ID C1_ROW...,Datasource = ['ODS.W_YFY_AV_TW_R']\n\nRelation...
1,C$_0W_YFY_IND_FIN_INFO_FS,"select FIN_INFO.SEQ C1_SEQ, FIN_INFO.ACC...",from ODS.TC$_0W_YFY_IND_FIN_INFO_FS FIN_INFO,Datasource = ['ODS.TC$_0W_YFY_IND_FIN_INFO_FS'...
2,OP_FACT_CHP_INVENTORY_ETH_PULP,"SELECT PERIOD_NAME,STOCK_DATE TDATE,ORG_CODE,'...",FROM W_FACTORY_INV_BALANCE_F UNION ALL SELECT ...,"Datasource = ['W_FACTORY_INV_BALANCE_F', 'W_FA..."
3,OP_FACT_CHP_INVENTORY_REDEFINE,"SELECT PERIOD_NAME,TDATE,ORG_CODE ,CASE ORG_...","FROM ( SELECT F.PERIOD_NAME,F.BALANCE_DATE T...","Datasource = ['W_YFY_INV_BALANCE_F', 'w_chp_it..."
4,OP_FACT_CHP_SALES_DETAILS,"SELECT '當月受訂' TYPE, --GREATEST(F.REQU...","from W_CHP_SALES_NOMANUAL_F F ,W_CHP_ORDER...","Datasource = ['W_CHP_SALES_NOMANUAL_F', 'W_CHP..."


In [49]:
bidb_view.iloc[1].lineage

"Datasource = ['ODS.TC$_0W_YFY_IND_FIN_INFO_FS']\n\nRelationship = []"

In [50]:
# def lineage_check(df):
#     for i, row in df[3:].iterrows():
#         print(f"#####{row.view_name}\n")
#         print(row. source)

#         print(row.lineage)

#         print("#####\n")

# lineage_check(bidb_view[bidb_view['view_name'] == 'OP_FACT_CHP_INVENTORY_REDEFINE'].text)

In [51]:
def extract_lists(input_string):
    # Initialize dictionary to hold the lists
    extracted_data = {}
    
    # Use regex to find the Datasource part
    datasource_match = re.search(r"Datasource\s*=\s*(\[.*?\])", input_string, re.DOTALL)
    if datasource_match:
        datasource_section = datasource_match.group(1).strip()
        extracted_data["Datasource"] = ast.literal_eval(datasource_section)
    
    # Use regex to find the Relationship part
    relationship_match = re.search(r"Relationship\s*=\s*(\[.*?\])", input_string, re.DOTALL)
    if relationship_match:
        relationship_section = relationship_match.group(1).strip()
        # Split the relationship section into individual items
        relationship_list = re.findall(r'\((.*?)\)', relationship_section, re.DOTALL)
        
        # Process each item in the relationship list
        processed_list = []
        for item in relationship_list:
            # Replace any variation of "FILTER" with "Filter"
            item = re.sub(r'^FILTER\s*:?', 'Filter:', item, flags=re.IGNORECASE)
            # Remove any newline characters and extra spaces
            item = re.sub(r'\s+', ' ', item.strip())
            processed_list.append(item)
        
        extracted_data["Relationship"] = processed_list

    return extracted_data

In [53]:
bidb_view.iloc[1]

view_name                            C$_0W_YFY_IND_FIN_INFO_FS
text         select   FIN_INFO.SEQ    C1_SEQ,  FIN_INFO.ACC...
source          from ODS.TC$_0W_YFY_IND_FIN_INFO_FS  FIN_INFO 
lineage      Datasource = ['ODS.TC$_0W_YFY_IND_FIN_INFO_FS'...
Name: 1, dtype: object

In [54]:
for idx, row in bidb_view.iterrows():
    # print(row)
    result = extract_lists(row.lineage)
    # print(result)
    datasource_list = result.get("Datasource", [])
    
    relationship_list = result.get("Relationship", [])
    print(relationship_list)



[]
[]
[]
['Filter: W_YFY_INV_BALANCE_F.ORGANIZATION_ID IN (290,291,305,286,287,288,289', 'JOIN: W_YFY_INV_BALANCE_F.ORGANIZATION_ID = w_chp_item_d.ORGANIZATION_ID', 'JOIN: W_YFY_INV_BALANCE_F.INVENTORY_ITEM_ID = w_chp_item_d.INVENTORY_ITEM_ID', 'JOIN: W_YFY_INV_BALANCE_F.ORGANIZATION_ID = W_YFY_ORG_D.ORG_ID', "Filter: w_chp_item_d.ITEM_NUMBER LIKE '4%'", "Filter: w_chp_item_d.CATEGORY_CODE = 'FG'", "Filter: W_YFY_INV_BALANCE_F.UOM = 'KG'", "Filter: w_chp_item_d.ITEM_DESC_TCH <> '漿塑盤'", "Filter: NOT(W_YFY_INV_BALANCE_F.SUB_INVENTORY IN ('RM','MTL','SFG','SFGW','FG-PP','FG-PP1'", 'JOIN: W_CHP_TS_BELONG_ORG_R.ORG_CODE(+', 'JOIN: W_CHP_TS_BELONG_ORG_R.SUBINVENTORY_CODE(+', "Filter: W_YFY_INV_BALANCE_F.PERIOD_NAME >= '2023/01'", 'UNION ALL', "Filter: W_YFY_INV_BALANCE_F.PERIOD_NAME = TO_CHAR(add_months(trunc(CURRENT_DATE,'mm'", "Filter: TO_CHAR(CURRENT_DATE,'dd'", 'UNION ALL', 'Filter: W_CHP_MMT_F.ORGANIZATION_ID IN (290,291,305,286,287,288,289', 'JOIN: W_CHP_MMT_F.ORGANIZATION_ID = w_chp_i

In [55]:
# print(bidb_view_raw.iloc[3].text)

In [56]:
print(bidb_view.iloc[2].source)

FROM W_FACTORY_INV_BALANCE_F UNION ALL SELECT PERIOD_NAME,STOCK_DATE TDATE,ORG_CODE,'紙漿' STOCK_TYPE, 4 STOCK_TYPE_SORT   ,INVENTORY_ITEM_ID   ,UOM,QTY FROM W_FACTORY_INV_F INV     ,(select max(stock_date) tdate from W_FACTORY_INV_F where ORG_CODE='ETH' AND MEMO='花蓮自製漿' AND PERIOD_NAME = to_char(CURRENT_DATE,'yyyy/mm') ) MS WHERE and INV.STOCK_DATE = MS.TDATE   AND INV.ORG_CODE='ETH' AND INV.MEMO='花蓮自製漿'


In [57]:
print(bidb_view.iloc[2].lineage)

Datasource = ['W_FACTORY_INV_BALANCE_F', 'W_FACTORY_INV_F']

[
(Filter: W_FACTORY_INV_F.ORG_CODE = 'ETH'),
(Filter: W_FACTORY_INV_F.MEMO = '花蓮自製漿'),
(JOIN: W_FACTORY_INV_F.STOCK_DATE = (SELECT max(stock_date) FROM W_FACTORY_INV_F WHERE ORG_CODE='ETH' AND MEMO='花蓮自製漿' AND PERIOD_NAME = to_char(CURRENT_DATE, 'yyyy/mm')) MS.TDATE)
]


In [58]:
print(bidb_view.iloc[3].lineage)

Datasource = ['W_YFY_INV_BALANCE_F', 'w_chp_item_d', 'W_YFY_ORG_D', 'W_CHP_TS_BELONG_ORG_R', 'W_CHP_MMT_F']

Relationship = [
    (Filter: W_YFY_INV_BALANCE_F.ORGANIZATION_ID IN (290,291,305,286,287,288,289)),
    (JOIN: W_YFY_INV_BALANCE_F.ORGANIZATION_ID = w_chp_item_d.ORGANIZATION_ID),
    (JOIN: W_YFY_INV_BALANCE_F.INVENTORY_ITEM_ID = w_chp_item_d.INVENTORY_ITEM_ID),
    (JOIN: W_YFY_INV_BALANCE_F.ORGANIZATION_ID = W_YFY_ORG_D.ORG_ID),
    (Filter: w_chp_item_d.ITEM_NUMBER LIKE '4%'),
    (Filter: w_chp_item_d.CATEGORY_CODE = 'FG'),
    (Filter: W_YFY_INV_BALANCE_F.UOM = 'KG'),
    (Filter: w_chp_item_d.ITEM_DESC_TCH <> '漿塑盤'),
    (Filter: NOT(W_YFY_INV_BALANCE_F.SUB_INVENTORY IN ('RM','MTL','SFG','SFGW','FG-PP','FG-PP1') OR W_YFY_INV_BALANCE_F.SUB_INVENTORY LIKE '%FGS%')),
    (JOIN: W_CHP_TS_BELONG_ORG_R.ORG_CODE(+) = W_YFY_ORG_D.ORG_CODE),
    (JOIN: W_CHP_TS_BELONG_ORG_R.SUBINVENTORY_CODE(+) = W_YFY_INV_BALANCE_F.SUB_INVENTORY),
    (Filter: W_YFY_INV_BALANCE_F.PERIOD_NAME >= 

In [61]:
# bidb_view.to_csv('./result/temp1.csv', index=False)

### Insert the datasource and relationship into Neo4j

In [10]:
# Create graph
graph = Graph(host="138.3.214.21", auth=("neo4j","yfy12345"))

In [8]:
bidb_view = pd.read_csv('./result/temp1.csv')
bidb_view

,view_name,text,source,lineage
0,C$_0W_YFY_AV_TW_R,"select ""C1_ROW_ID"",""C2_ORG_ID"",""C3_REF_AV_HEAD...",from ( select W_YFY_AV_TW_R.ROW_ID C1_ROW...,Datasource = ['ODS.W_YFY_AV_TW_R']\n\nRelation...
1,C$_0W_YFY_IND_FIN_INFO_FS,"select FIN_INFO.SEQ C1_SEQ, FIN_INFO.ACC...",from ODS.TC$_0W_YFY_IND_FIN_INFO_FS FIN_INFO,Datasource = ['ODS.TC$_0W_YFY_IND_FIN_INFO_FS'...
2,OP_FACT_CHP_INVENTORY_ETH_PULP,"SELECT PERIOD_NAME,STOCK_DATE TDATE,ORG_CODE,'...",FROM W_FACTORY_INV_BALANCE_F UNION ALL SELECT ...,"Datasource = ['W_FACTORY_INV_BALANCE_F', 'W_FA..."
3,OP_FACT_CHP_INVENTORY_REDEFINE,"SELECT PERIOD_NAME,TDATE,ORG_CODE ,CASE ORG_...","FROM ( SELECT F.PERIOD_NAME,F.BALANCE_DATE T...","Datasource = ['W_YFY_INV_BALANCE_F', 'w_chp_it..."
4,OP_FACT_CHP_SALES_DETAILS,"SELECT '當月受訂' TYPE, --GREATEST(F.REQU...","from W_CHP_SALES_NOMANUAL_F F ,W_CHP_ORDER...","Datasource = ['W_CHP_SALES_NOMANUAL_F', 'W_CHP..."


In [21]:
# check current nodes 
def neo4j_check(graph):
    query = """
    MATCH (a)-[r:FROM]->(b)
    RETURN a, r, b
    """

    # Execute the query and get the result
    result = graph.run(query)

    # Iterate over the result and print it
    if result == None:
        for record in result:
            print(f"Node A: {record['a']}")
            print(f"Relationship: {record['r']}")
            print(f"Node B: {record['b']}")
    else:
        print('no data in graph.')

neo4j_check(graph)

no data in graph.


In [22]:
# clean the nodes if the db is not empty
graph.run("MATCH ()-[r]->() DELETE r")

# Delete all nodes
graph.run("MATCH (n) DELETE n")

neo4j_check(graph)

no data in graph.


In [ ]:
result_dict = {}
for source_item in datasource_list:
    for rel_item in relationship_list:
        if source_item in rel_item:
            result_dict[source_item] = rel_item
            related_source = [rel_source for rel_source in datasource_list if rel_source != source_item]
            # num_related_source = len(related_source)
            source_table = Node("table", name=source_item)
            related_source_table = Node("table", name=related_source[0])

            property_dictionary = {}
            rel_item = rel_item.replace("(", "").replace(")", "").replace(",", "")
            if "Filter" in rel_item:
                property_dictionary['condition'] = rel_item
            elif "JOIN" in rel_item:
                property_dictionary['merge'] = rel_item

            relationship = Relationship(source_table, "FROM", related_source_table, **property_dictionary)
